In [83]:
import os
import numpy as np
import lsst.sims.skybrightness as sb
import lsst.sims.ocs
from lsst.sims.utils import altAzPaFromRaDec, ObservationMetaData

In [84]:
field_dict = {}
with open("tessellationFields") as fh:
    field_id = 1
    for line in fh:
        line = line.strip()
        # Skip blank line
        if not line:
            continue
        # Skip comment lines
        if line.startswith("#"):
            continue
        values = (float(x) for x in line.split())
        field = lsst.sims.ocs.configuration.Field(field_id, 3.5, *values)
        field_dict[field_id] = field
        field_id += 1

In [96]:
# Default is LSST observatory and preciseAltAz off but give back LSST filter magnitudes 
sm = sb.SkyModel(mags=True)

In [97]:
mjd = 59581.041666666664
ra = np.array([field.ra for _, field in field_dict.iteritems()])
dec = np.array([field.dec for _, field in field_dict.iteritems()])
obs = ObservationMetaData(mjd=mjd)

alt, az, pa = altAzPaFromRaDec(ra, dec, obs)

UTC 5.958104e+04 is outside of IERS table for UT1-UTC.
Returning UT1 = UTC for lack of a better idea
UTC 5.958104e+04 is outside of IERS table for UT1-UTC.
Returning UT1 = UTC for lack of a better idea


In [98]:
alt

array([ 30.34537627,  28.62848452,  30.67382062, ..., -29.23353408,
       -27.18671828, -28.90499397])

In [99]:
alt_mask = np.where(alt > 20.0)
print(alt_mask)
up_ra = ra[alt_mask]
up_dec = dec[alt_mask]
up_alt = alt[alt_mask]
up_az = az[alt_mask]

(array([   0,    1,    2, ..., 4304, 4312, 4322]),)


In [100]:
sm.setRaDecMjd(up_ra, up_dec, mjd, degrees=True)
print(sm.mjd, sm.sunAlt, sm.sunAz, sm.moonPhase, sm.moonAlt, sm.moonAz)
mags = sm.returnMags()

(59581.041666666664, -0.24299991130828857, 4.057347774505615, 0.8692556023597717, -0.3459267020225525, 3.885237216949463)


In [81]:
# For each RA,DEC you get a ugrizy map
i_filt = 3
bad_count = 0
for ra, dec, alt, az, fmag in zip(up_ra, up_dec, up_alt, up_az, mags):
    i_mag = fmag[i_filt]
    if i_mag == np.Inf:
        bad_count += 1
    print(ra, dec, alt, az, i_mag)
print("Number of bad fields: {} out of {}".format(bad_count, up_ra.size))

(0.0, -90.0, 30.345376266467646, 180.09527155034436, 19.396177188454679)
(180.0, -87.568555000000003, 28.628484517578688, 178.11934388920477, 19.15310525832464)
(324.000429, -87.568550000000002, 30.673820621229673, 182.89104307183246, 19.219970930074826)
(35.999571000000003, -87.568550000000002, 32.740622992961619, 180.57330183382987, 19.319853975106021)
(252.001105, -87.568546999999995, 28.16512868052487, 181.32440563203664, 19.107729888247263)
(107.998895, -87.568546999999995, 31.443857881276394, 177.56835942640558, 19.28284278977846)
(215.99982199999999, -85.272913000000003, 25.685022525096972, 179.22856568825128, 18.977436415427476)
(144.00017800000001, -85.272913000000003, 29.539288072047285, 174.72070079943745, 19.21670247859991)
(0.0, -85.272891999999999, 33.592759707175048, 184.14238846838666, 19.33543521518342)
(288.00050800000002, -85.272852, 28.077125181878454, 184.84562592146685, 19.078901046445964)
(71.999492000000004, -85.272852, 34.548933141812903, 177.53909462061873, 19

In [89]:
dir(sm)

['Observatory',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_initPoints',
 'airglow',
 'airmassLimit',
 'components',
 'interpObjs',
 'interpSky',
 'lowerAtm',
 'mags',
 'mergedSpec',
 'moon',
 'npix',
 'paramsSet',
 'preciceAltAz',
 'returnMags',
 'returnWaveSpec',
 'scatteredStar',
 'setComponents',
 'setParams',
 'setRaDecMjd',
 'telescope',
 'twilight',
 'upperAtm',
 'zodiacal']

In [90]:
print(sm.moon)

True


In [91]:
print(sm.sunAlt, sm.sunAz)

AttributeError: 'SkyModel' object has no attribute 'sunAlt'